<a href="https://colab.research.google.com/github/NLP-KazNU/CSE-morphology-model-of-the-Uzbek-language/blob/main/AudioTranscriptionWav2vec2Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 19.7 MB/s 
     |████████████████████████████████| 7.6 MB 55.7 MB/s 
     |████████████████████████████████| 182 kB 73.8 MB/s 


In [ ]:
import librosa
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer

In [ ]:
tokenizer = Wav2Vec2Tokenizer.from_pretrained("Bauyrjan/wav2vec2-kazakh")
model = Wav2Vec2ForCTC.from_pretrained("Bauyrjan/wav2vec2-kazakh")

Downloading:   0%|          | 0.00/445 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:370: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:755: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Proce

Downloading:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

In [ ]:
file_path = "test3.wav"
speech, rate = librosa.load(file_path,sr=16000)

In [ ]:
%%time
input_values = tokenizer(speech, return_tensors = 'pt').input_values

#Store logits (non-normalized predictions)
logits = model(input_values).logits

#Store predicted id's
predicted_ids = torch.argmax(logits, dim =-1)

#decode the audio to generate text
transcript = tokenizer.decode(predicted_ids[0])

CPU times: user 6 s, sys: 636 ms, total: 6.63 s
Wall time: 6.62 s


In [ ]:
print(transcript)

абай қазақ әдебиетінің негізін қалаған ағартушы философ


In [ ]:
with open("test.txt", "w") as f:
    f.write(transcript)

In [ ]:
import re
import os
import xlrd, xlwt


#------------------------- WORD in SL = STEM + ENDING in SL -------------------------#

def splitting_by_words(text):
    result = re.findall("\w+\'?\\w+", text)
    return result


def sorting_endings(endings_file_name):
    endings_wb = xlrd.open_workbook(endings_file_name)
    endings_sh = endings_wb.sheet_by_index(0)
    endings = []
    for rownum in range(endings_sh.nrows-1):
        ending = endings_sh.cell(rownum+1,1).value
        if '\ufeff' or '-' in ending:
            ending = ending.replace('\ufeff', '')
            ending = ending.replace('-', '')
        endings.append(ending)

    sorted_endings = sorted(endings, key=len, reverse=True)
    ###print(sorted_endings)

    return sorted_endings


def stem(word, endings, stems_file_name):
    stems_wb = xlrd.open_workbook(stems_file_name)
    stems_sh = stems_wb.sheet_by_index(0)
    stems = []
    for rownum in range(stems_sh.nrows-1):
        stem = stems_sh.cell(rownum+1,0).value
        if '\ufeff' in stem:
            stem = stem.replace('\ufeff', '')
        stems.append(stem)

    stems_list = sorted(stems, key=len, reverse=True)
    ###print(stems_list)
    
    word_len = len(word)
    min_len_of_word = 2

    if word_len > min_len_of_word:
        n = word_len - min_len_of_word

        if word in stems_list:
            rez_stem = word
            rez_ending = ""

        else:
            i = n+1
            while i > 0:
                word_ending = word[word_len - (i-1):]
                stem = word[:word_len-len(word_ending)]                         
                for ending in endings:
                    if word_ending == ending:
                        if stem in stems_list:
                            rez_stem = stem
                            rez_ending = word_ending
                            i = 0
                if word_ending == '':
                    if stem in stems_list:
                        rez_stem = stem
                        rez_ending = word_ending
                        i = 0
                    else:
                        j = n+1
                        while j > 0:
                            word_ending = word[word_len - (j-1):]
                            stem = word[:word_len-len(word_ending)]
                            for ending in endings:
                                if word_ending == ending:
                                    rez_stem = stem
                                    rez_ending = word_ending
                                    j = 0
                                    i = 0
                            if word_ending == '':
                                rez_stem = word
                                rez_ending = word_ending
                                j = 0
                                i = 0
                            j = j-1
                i = i-1

    else:
        rez_stem = word
        rez_ending = ""
                        
    return rez_stem, rez_ending
    

def stemming(tfile_name, endings, stopwords_file_name, stems_file_name):
    text_file = open(tfile_name, 'r', encoding="utf-8")
    text_file = text_file.read()

    stopwords_wb = xlrd.open_workbook(stopwords_file_name)
    stopwords_sh = stopwords_wb.sheet_by_index(0)
    stopwords = []
    for rownum in range(stopwords_sh.nrows-1):
        stopword = stopwords_sh.cell(rownum+1,0).value
        if '\ufeff' in stopword:
            stopword = stopword.replace('\ufeff', '')
        stopwords.append(stopword)
    stopwords_list = sorted(stopwords, key=len, reverse=True)
    ###print(stopwords_list)
        
    text = splitting_by_words(text_file)
    ###print("text=",text)
    res_text = []

    rim_cifry = ['i', 'ii', 'iii', 'iv', 'v', 'vi', 'vii', 'viii', 'ix', 'x', 'xi', 'xii', 'xiii', 'xiv', 'xv', 'xvi', 'xvii', 'xviii', 'xix', 'xx', 'xxi', 'xxiv']

    for word in text:
        word = word.lower()
        if word not in res_text:
            if word.isnumeric() or word in rim_cifry:
                continue
            res_text.append(word)

    result_words  = [word for word in res_text if word not in stopwords_list]
    ##print("rez=",result_words)
    
    text_by_stemending = {}
    for word in result_words:
        stemm, endingg = stem(word, endings, stems_file_name)
        text_by_stemending.update({word: [stemm, endingg]})
    ##print(text_by_stemending)
       
    return text_by_stemending


#------------------------- STEM + ENDING in SL = STEM + ENDING in TL = WORD in TL -------------------------#

def match_by_table(endings_file, stems_file, stemsandendings_insl):
    stems_wb = xlrd.open_workbook(stems_file)
    stems_sh = stems_wb.sheet_by_index(0)
    stems = {}
    for rownum in range(stems_sh.nrows-1):
        stem_in_sl = stems_sh.cell(rownum+1,0).value
        stem_in_tl = stems_sh.cell(rownum+1,1).value
        stems.update({stem_in_sl: stem_in_tl})
    ###print("Stems ", stems)

    endings_wb = xlrd.open_workbook(endings_file)
    endings_sh = endings_wb.sheet_by_index(0)
    endings = {}
    for rownum in range(endings_sh.nrows-1):
        ending_in_sl = endings_sh.cell(rownum+1,1).value
        ending_in_sl = ending_in_sl.replace('-', '')
        ending_in_tl = endings_sh.cell(rownum+1,4).value
        endings.update({ending_in_sl: ending_in_tl})
    ###print("Endings ", endings)

    matched_words = {}
    for word_in_sl in stemsandendings_insl.keys():
        ###print("*****", word_in_sl, "*****")
        stem_in_sl = stemsandendings_insl[word_in_sl][0]
        ending_in_sl = stemsandendings_insl[word_in_sl][1]
        #print(stem_in_sl, ending_in_sl)
        if stem_in_sl in stems.keys():
            stem_in_tl = stems[stem_in_sl]
            ###print(stem_in_tl)
        else:
            stem_in_tl = stem_in_sl
            ###print(stem_in_tl)
        if ending_in_sl in endings.keys():
            ending_in_tl = endings[ending_in_sl]
            ###print(ending_in_tl)
        else:
            ending_in_tl = ending_in_sl
            ###print(ending_in_tl)
        word_in_tl = stem_in_tl + ending_in_tl
        matched_words.update({word_in_sl: word_in_tl})
        #print(word_in_sl, word_in_tl)

    return matched_words

def replace_sltotl(text_file, matchings, stopwords_file_name):
    stopwords_wb = xlrd.open_workbook(stopwords_file_name)
    stopwords_sh = stopwords_wb.sheet_by_index(0)
    stopwords = {}
    for rownum in range(stopwords_sh.nrows-1):
        stopword_in_sl = stopwords_sh.cell(rownum+1,0).value
        stopword_in_tl = stopwords_sh.cell(rownum+1,1).value
        stopwords.update({stopword_in_sl: stopword_in_tl})
    ###print(stopwords)

    text_file = open(text_file, 'r', encoding="utf-8")
    text_file = text_file.read()
    punctuations = ['.', ',', '?', '!', ':', ';', '(', ')', '"', '«', '»']
    for punctn in punctuations:
        text_file = text_file.replace(punctn, " "+punctn+" ")
    text_file = text_file.replace("  ", " ")
    print("TEXT in SL")
    print(text_file)
    
    for word in matchings:
        ###print(word)
        if f"{word} " in text_file.lower():
            ###print(matchings[word])
            text_file = text_file.lower().replace(f"{word} ", matchings[word]+" ")

    for stopword in stopwords:
        ###print(stopword)
        if f" {stopword} " in text_file.lower():
            ###print(stopwords[stopword])
            text_file = text_file.lower().replace(f" {stopword} ", " "+stopwords[stopword]+" ")
            
    print("\nTEXT in TL")
    translated_text = text_file
    
    return translated_text

stopwords_file_name = "qaz-tat-stopwords.xlsx"

endings_file_name = "qaz-tat-tab.xlsx"
endings = sorting_endings(endings_file_name)
###print(endings)

text_file_name = "test.txt"
stems_file_name = "qaz-tat-stems.xlsx"
text_by_stemending = stemming(text_file_name, endings, stopwords_file_name, stems_file_name)
###print(text_by_stemending)

matchings = match_by_table(endings_file_name, stems_file_name, text_by_stemending)
###print(matchings)

translated_text = replace_sltotl(text_file_name, matchings, stopwords_file_name)
print(translated_text)

TEXT in SL
абай қазақ әдебиетінің негізін қалаған ағартушы философ

TEXT in TL
абай казах әдәбиятына нигез қалағаны мәгърифәтче философ


In [ ]:
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/tokens .
    
Token: 
Add token as git credential? (Y/n) y
Token is valid.
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper

In [ ]:
!wget https://issai.nu.edu.kz/wp-content/uploads/2022/03/parallelwavegan_male2_checkpoint.zip

--2022-11-23 13:18:20--  https://issai.nu.edu.kz/wp-content/uploads/2022/03/parallelwavegan_male2_checkpoint.zip
Resolving issai.nu.edu.kz (issai.nu.edu.kz)... 178.91.253.144
Connecting to issai.nu.edu.kz (issai.nu.edu.kz)|178.91.253.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15610294 (15M) [application/zip]
Saving to: ‘parallelwavegan_male2_checkpoint.zip’

parallelwavegan_mal 100%[===================>]  14.89M  3.99MB/s    in 3.7s    

2022-11-23 13:18:26 (3.99 MB/s) - ‘parallelwavegan_male2_checkpoint.zip’ saved [15610294/15610294]



In [ ]:
!unzip parallelwavegan_male2_checkpoint.zip

Archive:  parallelwavegan_male2_checkpoint.zip
   creating: parallelwavegan_male2_checkpoint/
  inflating: parallelwavegan_male2_checkpoint/config.yml  
  inflating: parallelwavegan_male2_checkpoint/checkpoint-400000steps.pkl  


In [ ]:
!wget https://issai.nu.edu.kz/wp-content/uploads/2022/03/kaztts_male2_tacotron2_train.loss.ave.zip

--2022-11-23 13:18:32--  https://issai.nu.edu.kz/wp-content/uploads/2022/03/kaztts_male2_tacotron2_train.loss.ave.zip
Resolving issai.nu.edu.kz (issai.nu.edu.kz)... 178.91.253.144
Connecting to issai.nu.edu.kz (issai.nu.edu.kz)|178.91.253.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 107280461 (102M) [application/zip]
Saving to: ‘kaztts_male2_tacotron2_train.loss.ave.zip’

kaztts_male2_tacotr 100%[===================>] 102.31M  7.66MB/s    in 15s     

2022-11-23 13:18:50 (6.71 MB/s) - ‘kaztts_male2_tacotron2_train.loss.ave.zip’ saved [107280461/107280461]



In [ ]:
!unzip kaztts_male2_tacotron2_train.loss.ave.zip

Archive:  kaztts_male2_tacotron2_train.loss.ave.zip
 extracting: meta.yaml               
 extracting: exp/tts_train_raw_char/config.yaml  
 extracting: exp/tts_train_raw_char/train.loss.ave_5best.pth  
 extracting: exp/tts_train_raw_char/images/gpu_max_cached_mem_GB.png  
 extracting: exp/tts_train_raw_char/images/iter_time.png  
 extracting: exp/tts_train_raw_char/images/optim0_lr0.png  
 extracting: exp/tts_train_raw_char/images/train_time.png  
 extracting: exp/tts_train_raw_char/images/mse_loss.png  
 extracting: exp/tts_train_raw_char/images/forward_time.png  
 extracting: exp/tts_train_raw_char/images/backward_time.png  
 extracting: exp/tts_train_raw_char/images/loss.png  
 extracting: exp/tts_train_raw_char/images/l1_loss.png  
 extracting: exp/tts_train_raw_char/images/optim_step_time.png  
 extracting: exp/tts_train_raw_char/images/attn_loss.png  
 extracting: exp/tts_train_raw_char/images/bce_loss.png  
 extracting: exp/tts_stats_raw_char/train/feats_stats.npz  


In [ ]:
!pip install parallel_wavegan

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 496 kB/s 
     |████████████████████████████████| 125 kB 62.8 MB/s 
  Created wheel for parallel-wavegan: filename=parallel_wavegan-0.5.5-py3-none-any.whl size=72466 sha256=3a3b1a0222170b6cace8bf507864e01cdd0166f548cb254c9c31c26694265aec
  Stored in directory: /root/.cache/pip/wheels/67/cb/04/5bdedaaec4d6bf6dd46df360c30515ae58364cfa48e2f953bb
  Created wheel for kaldiio: filename=kaldiio-2.17.2-py3-none-any.whl size=24471 sha256=43d2542ffb9ec7af0af9244fdd8c4858635467394aef3cdebe47e340f4ba5e1c
  Stored in directory: /root/.cache/pip/wheels/04/07/e8/45641287c59bf6ce41e22259f8680b521c31e6306cb88392ac
Successfully built parallel-wavegan kaldiio


In [ ]:
!pip install espnet[torch]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 28.7 MB/s 
     |████████████████████████████████| 214 kB 71.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.3 MB 61.9 MB/s 
     |████████████████████████████████| 86 kB 6.1 MB/s 
     |████████████████████████████████| 73 kB 2.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.3 MB 56.8 MB/s 
     |████████████████████████████████| 41 kB 471 kB/s 
     |████████████████████████████████| 3.1 MB 56.2 MB/s 
     |████████████████████████████████| 235 kB 60.5 MB/s 
     |████████████████████████████████| 180 kB 68.3 MB/s 
  Created wheel for fast-bss-eval: filename=fast_bss_eval-0.1.3-py3-none-an

In [ ]:
import ipa_convert

dispatcher = {'kazakh' : ipa_convert.kazakh_to_ipa, 
              'turkish' : ipa_convert.turkish_to_ipa,
              'azerbaijani': ipa_convert.azerbaijani_to_ipa,
              'kyrgyz' : ipa_convert.kyrgyz_to_ipa,              
              'uzbek' : ipa_convert.uzbek_to_ipa,
              'turkmen' : ipa_convert.turkmen_to_ipa,
              'tatar' : ipa_convert.tatar_to_ipa,
              'bashkir' : ipa_convert.bashkir_to_ipa,
              'sakha' : ipa_convert.sakha_to_ipa,
              'uyghur' : ipa_convert.uyghur_to_ipa,              
              'tts_sent' : ipa_convert.ipa_to_kazakh}

def call_func(x, func):
    try:
        return dispatcher[func](x)
    except:
        return "Invalid function"

def normalization(x, lang="kazakh"):
    ipa_text = call_func(x, lang)
    kz_text = call_func(ipa_text, 'tts_sent')
    return kz_text

In [ ]:
from parallel_wavegan.utils import load_model
from espnet2.bin.tts_inference import Text2Speech
from scipy.io.wavfile import write
# from utils import normalization
import torch

fs = 22050
vocoder_checkpoint="parallelwavegan_male2_checkpoint/checkpoint-400000steps.pkl" ### specify vocoder path
vocoder = load_model(vocoder_checkpoint).to("cuda").eval()
vocoder.remove_weight_norm()

## specify path to the main model(transformer/tacotron2/fastspeech) and its config file
config_file = "exp/tts_train_raw_char/config.yaml"
model_path = "exp/tts_train_raw_char/train.loss.ave_5best.pth"

text2speech = Text2Speech(
    config_file,
    model_path,
    device="cuda", ## if cuda not available use cpu
    # Only for Tacotron 2
    threshold=0.5,
    minlenratio=0.0,
    maxlenratio=10.0,
    use_att_constraint=True,
    backward_window=1,
    forward_window=3,
    # Only for FastSpeech & FastSpeech2
    speed_control_alpha=1.0,
)
text2speech.spc2wav = None  # Disable griffin-lim

# text = """Iqlimshunos Erkin Abdulahatov agar yo‘l qurilishida tartiblar ketma-ketligida ishlar to‘g‘ri bajarilmasa, barpo etilgan infratuzilma birinchi kuchli tabiat hodisasiga dosh bera olmasligini ta’kidlaydi."""

# text = """Ақын саяси қызметі үшін 1870 жылдары Санкт-Петербургтен Семейге айдалып келген Михаэлиспен, 80-жылдарда орыс демократтары Н. И. Долгополов, А. А.Леонтьевпен танысады."""
# text = """татарларның милли теле, Татарстанның дәүләт теле, Русиядә таралу буенча икенче тел"""
### available options are: kazakh, turkish, azerbaijani, kyrgyz, uzbek, turkmen, tatar, bashkir, sakha, uyghur
text = translated_text
lang = "tatar"
 
text = normalization(text, lang)
with torch.no_grad():
    c_mel = text2speech(text)['feat_gen']
    wav = vocoder.inference(c_mel)
write("result.wav", fs, wav.view(-1).cpu().numpy())

In [ ]:
import IPython.display as ipd

In [ ]:
ipd.display(ipd.Audio(wav.view(-1).cpu().numpy(), rate=fs))

In [ ]:
!nvidia-smi

Wed Nov 23 13:14:03 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------